In [60]:
%matplotlib inline
%matplotlib notebook

In [61]:
import os 
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading In Data

In [62]:
PATH = os.path.abspath(os.path.join(os.path.dirname("."), "Data"))
FINAL_DATA = os.path.join(PATH, "finaldataframecopy_wWeather.csv")

FINAL_DATA

'/Users/wrangamujadidi/Desktop/Capstone/capstone_opioid/Data/finaldataframecopy_wWeather.csv'

In [63]:
data = pd.read_csv(FINAL_DATA)

In [64]:
data.head()

,HexagonNumber,HexandMonth,Month,Under20,20to29,30to39,40to49,50to59,60to69,70andabove,...,MiscCals,OverdoseCalls,PropertyCalls,QOLCalls,TrafficCalls,ViolentCalls,AverageTemperature,AverageHumidity,AveragePrecipitation,Unemployment
0,1,1-1,1,0,0,0,0,0,0,0,...,0,0,0,5,5,0,37.4075,0.541429,0.00025,0.274
1,1,1-2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.274
2,1,1-3,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.274
3,1,1-4,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.274
4,1,1-5,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,37.4075,0.541429,0.00025,0.274


In [65]:
#Drop the columns for HexandMonth and TotalNarcan.  HEXandMonth is not needed and TotalNarcan may cause bias because
#the Narcan Administered already accounts for the TotalNarcan for the month. 

data = data.drop(['HexandMonth'], axis = 1)
data = data.drop(['TotalNarcan'], axis = 1)


In [66]:
data.columns

Index(['HexagonNumber', 'Month', 'Under20', '20to29', '30to39', '40to49',
       '50to59', '60to69', '70andabove', 'NarcanAdministered', 'Male',
       'Female', 'MPDCalls', 'FEMSCalls', 'MedicalCalls', 'MiscCals',
       'OverdoseCalls', 'PropertyCalls', 'QOLCalls', 'TrafficCalls',
       'ViolentCalls', 'AverageTemperature', 'AverageHumidity',
       'AveragePrecipitation', 'Unemployment'],
      dtype='object')

# Test Train Split

In [67]:
from sklearn.model_selection import train_test_split as tts 

features = data [[
     'HexagonNumber', 'Month', 'Under20', '20to29', '30to39', '40to49',
       '50to59', '60to69', '70andabove', 'Male',
       'Female', 'MPDCalls', 'FEMSCalls', 'MedicalCalls', 'MiscCals',
       'OverdoseCalls', 'PropertyCalls', 'QOLCalls', 'TrafficCalls',
       'ViolentCalls', 'AverageTemperature', 'AverageHumidity',
       'AveragePrecipitation', 'Unemployment'
]]

X = features

y = data['NarcanAdministered']

print(X.shape)
print(y.shape)

(9444, 24)
(9444,)


In [68]:
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [69]:
X_train

,HexagonNumber,Month,Under20,20to29,30to39,40to49,50to59,60to69,70andabove,Male,...,MiscCals,OverdoseCalls,PropertyCalls,QOLCalls,TrafficCalls,ViolentCalls,AverageTemperature,AverageHumidity,AveragePrecipitation,Unemployment
8616,719,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,55.878214,0.458571,0.000000,0.028000
2887,241,8,0,0,0,0,0,0,0,0,...,2,0,2,5,1,0,58.126429,0.493571,0.006225,0.045000
7743,646,3,0,0,0,0,0,0,0,0,...,1,0,0,5,1,1,64.988571,0.627857,0.002068,0.158000
2589,216,10,0,0,0,0,0,0,0,0,...,22,1,2,28,27,26,58.126429,0.493571,0.006225,0.542000
4130,345,4,1,0,1,0,2,1,0,4,...,35,1,29,93,41,19,78.354286,0.562857,0.000039,0.060000
9090,758,7,0,0,0,0,0,0,0,0,...,6,0,3,15,13,0,43.518571,0.563929,0.001679,0.034500
5800,484,5,0,0,0,0,0,0,0,0,...,28,3,20,54,32,12,84.513929,0.580714,0.000704,0.141800
3472,290,1,0,0,0,0,0,0,0,0,...,26,4,16,67,21,16,63.496429,0.676786,0.007686,0.304000
4024,336,2,0,1,0,0,0,0,0,1,...,0,0,0,4,4,4,78.354286,0.562857,0.000039,0.009000
5702,476,7,0,0,0,1,0,1,0,2,...,0,1,1,0,0,0,84.513929,0.580714,0.000704,0.026000


# Decision Tree

In [70]:
from sklearn.metrics import classification_report

In [71]:
# Import the model form 
from sklearn.tree import DecisionTreeClassifier 

model = DecisionTreeClassifier() 
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1566
          1       1.00      1.00      1.00       323

avg / total       1.00      1.00      1.00      1889



In [72]:
model = DecisionTreeClassifier(max_depth=5) 
model.fit(X_test, y_test)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1566
          1       1.00      1.00      1.00       323

avg / total       1.00      1.00      1.00      1889



# Random Forest 

In [73]:
from sklearn.ensemble import RandomForestClassifier

In [74]:
# We'll select 50 trees and opt for 'out-of-bag' samples to estimate the generalization error.
rf = RandomForestClassifier(n_estimators=50, oob_score=True)

In [75]:
# Next split up the data with the 'train test split' method in the Cross Validation module
X_train, X_test, y_train, y_test = tts(X,y, test_size=0.2)

# ...and then run the 'fit' method to build a forest of trees
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [76]:
rf.score(X_test, y_test)

1.0

# SVC

In [77]:
from sklearn.svm import SVC

model = SVC() 
model.fit(X_test, y_test)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99      1546
          1       1.00      0.89      0.94       343

avg / total       0.98      0.98      0.98      1889



# K Means Clustering 

In [78]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
K = range(1,10)
meandistortions = []

for k in K:

    elbow = KMeans(n_clusters=k, n_jobs=-1, random_state=1)
    elbow.fit(X)
    meandistortions.append(sum(np.min(euclidean_distances(X, elbow.cluster_centers_), axis=1)) / X.shape[0])

    
plt.plot(K, meandistortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Average distortion')
plt.title('Selecting k with the Elbow Method')
plt.show()

In [ ]:
import seaborn as sns
sns.set(style="ticks")

df = pd.read_csv(FINAL_DATA)
sns.pairplot(df, hue="NarcanAdministered")